<a href="https://colab.research.google.com/github/Azufendei/Quantum-Computing/blob/main/LAB1QSS2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit[visualization]==1.1.0
# Use the following if you are on MacOS/zsh
#!pip install 'qiskit[visualization]'==1.1.0
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 5.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=ab55eb7b9f44fe09674d0d783c413b86c755a7767f8a0342af499018ef530b5d
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
### Save API Token, if needed
%set_env QXToken=5cf69d36ae777d8d62f5c06ab7f728fd79a227bcaedd996fb8177ce50c71927716a688b502b9ebe02ec8de78ac9d2ede31e1418d0fa5a98dab9ec3f9e0331aec
# Make sure there is no space between the equal sign
# and the beginning of your token
# Make sure you do NOT ADD QUOTATION MARKS!!!!

env: QXToken=5cf69d36ae777d8d62f5c06ab7f728fd79a227bcaedd996fb8177ce50c71927716a688b502b9ebe02ec8de78ac9d2ede31e1418d0fa5a98dab9ec3f9e0331aec


In [ ]:
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [ ]:
# get an abstract quantum circuit from Qiskit's library of circuits
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [ ]:
# Hint: see https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit
def get_qc_characteristics(qc):
    # Your work goes here!
    # determine the quantum circuit depth of `qc` and assign it to `depth`
    depth = None
    # determine the number of qubits in `qc` and assign it to `num_qubits`
    num_qubits = None
    # determine the operations in `qc` and assign it to `ops`
    ops = None
    # determine the number of n-qubit operations (with n larger than 1) in `qc` and assign it to `num_multi_qubit_ops`,
    num_multi_qubit_ops = None
    # do not modify the next line
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [ ]:
def get_qc_characteristics(qc):
    # Determine the quantum circuit depth
    depth = qc.depth()

    # Determine the number of qubits in the quantum circuit
    num_qubits = qc.num_qubits

    # Determine the operations in the quantum circuit
    ops = {}
    for instruction, _, _ in qc.data:
        name = instruction.name
        if name in ops:
            ops[name] += 1
        else:
            ops[name] = 1

    # Determine the number of multi-qubit operations (with n larger than 1)
    num_multi_qubit_ops = sum(1 for instr, qargs, cargs in qc.data if len(qargs) > 1)

    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}




In [ ]:
# grade_lab1_ex1(get_qc_characteristics)

In [ ]:
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [ ]:
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [ ]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [ ]:
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
                                                                                                                                                                                                               ┌───┐   ┌───────────┐   ┌───┐     ┌──────────┐                              ┌───┐    ┌───────────┐    ┌───┐     ┌──────────┐                  ┌───┐┌───────────┐                               ┌───┐ ┌──────────┐                                          ┌───┐ ┌──────────┐              ┌───┐┌─────────┐                  ┌───┐    ┌──────────┐    ┌───┐     ┌──────────┐    ┌────┐    ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                               
q_4 -> 0 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├───┤ Rz(-π/64) ├───┤ X ├─────┤ Rz(π/64) ├──────────────────────────────┤ X ├────┤ Rz(-π/32) ├────┤ X ├─────┤ Rz(π/32) ├──────────────────┤ X ├┤ Rz(-π/16) ├───────────────────────────────┤ X ├─┤ Rz(π/16) ├──────────────────────────────────────────┤ X ├─┤ Rz(-π/8) ├──────────────┤ X ├┤ Rz(π/8) ├──────────────────┤ X ├────┤ Rz(-π/4) ├────┤ X ├─────┤ Rz(3π/4) ├────┤ √X ├────┤ Rz(3.0434) ├────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■────────■───────────────────────────────────────■─────────────────────────■─────────────────────■─────────────────────■──────────────────────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                                               └─┬─┘   └───────────┘   └─┬─┘     └──────────┘                              └─┬─┘    └───────────┘    └─┬─┘     └──────────┘                  └─┬─┘└───┬───┬───┘┌────────────┐                 └─┬─┘ └──┬───┬───┘ ┌───────────┐                            └─┬─┘ └──┬───┬───┘┌────────────┐└─┬─┘└──┬───┬──┘┌───────────┐     └─┬─┘    └──────────┘    └─┬─┘     └──┬───┬───┘┌───┴────┴───┐└────────────┘                  ┌───┐┌───────────┐                      ┌───┐┌───────────┐                │  ┌───┐ ┌──────────┐   ┌───┐      │        │       ┌───────────┐    ┌───┐          │       ┌──────────┐    ┌─┴─┐    ┌───────────┐┌─┴─┐┌──────────┐       │               ┌───┐┌──────────┐  │              ┌───┐┌─────────┐    ┌───┐    ┌──────────┐   ┌───┐    ┌──────────┐   ┌────┐   ┌──────────┐                                                              
q_1 -> 1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────────┼───────────────────────────────────────────────────┼─────────────────────────┼───────────────────────────────────────┼──────┤ X ├────┤ Rz(-π/512) ├───────────────────┼──────┤ X ├─────┤ Rz(π/512) ├──────────────────────────────┼──────┤ X ├────┤ Rz(-π/256) ├──┼─────┤ X ├───┤ Rz(π/256) ├───────┼────────────────────────┼──────────┤ X ├────┤ Rz(-π/128) ├────────────────────────────────┤ X ├┤ Rz(π/128) ├──────────────────────┤ X ├┤ Rz(-π/64) ├────────────────┼──┤ X ├─┤ Rz(π/64) ├───┤ X ├──────┼────────┼───────┤ Rz(-π/32) ├────┤ X ├──────────┼───────┤ Rz(π/32) ├────┤ X ├────┤ Rz(-π/16) ├┤ X ├┤ Rz(π/16) ├─────

In [ ]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'sx': 10, 'cx': 90}
  Number of multi-qubit Operations: 90


In [ ]:


# assign a 10-qubit linear CouplingMap to the variable cm
cm = CouplingMap.from_line(10)

# transpile the synthesized quantum circuit for the given coupling map
qc_routed = transpile(qc, coupling_map=cm)


In [ ]:
# grade_lab1_ex2(qc_routed)

In [ ]:

pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)

In [ ]:
# # Initialize the StagedPassManager
# pm_staged = StagedPassManager()

# # Replace the n-qubit QFT operation with its decomposition in two-qubit gates
# pm_staged.init = generate_unroll_3q(None)

# # Initialize the layout stage with an empty pass manager
# pm_staged.layout = PassManager()

# # Set a 'trivial' initial layout
# pm_staged.layout += TrivialLayout(cm)

# # Embed the circuit according to the coupling map
# pm_staged.layout += generate_embed_passmanager(cm)


In [ ]:

pm_staged.routing = PassManager(SabreSwap(cm))
qc_routed = pm_staged.run(qc)
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)

Quantum circuit characteristics
  Depth: 69
  Number of qubits: 10
  Operations: {'h': 10, 'cp': 45, 'swap': 81}
  Number of multi-qubit Operations: 126


q_0 -> 0 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────────────────────────────────────────────────■───────────────────────────X───────────X────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                         │                                                 │P(π/64)                    │           │                                                                                                                                                                                                                    
q_1 -> 1 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X──────X─────────X─────────────────────────────────X─────■─────────■──────────X──────X───────X───X──────────────────────────────────■───────────────────────X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                  │                │                                 │               │P(π/128)  │              │                                      │P(π/64)                │                                                                                                                                                         
q_2 -> 2 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────X──■─────────────X──────■────────────────────X─────X───────────────■──────────X──■───────────X───X───────────────────────────────X──■─────────■─────────────X─────X────────────X──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                       │P(π/128)     │P(π/256)            │P(π/512)            │                                   │P(π/256)       │                               │            │P(π/128)           │            │                                                                                                                                      
q_3 -> 3 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────X──────■─────────────■────────────────────■─────────────X──────X───────────────────────────────────■───────────────X──X─────────────────────X──────X────────────■───────────────────X─────X──────X───X────────────────────────────────────────■──────────────────────────────────────────■──────────────────────────────────────────────
                                                                                                                                   │P(π/64)     │                                                       │                                                             │                     │                                             │          │                                        │P(π/8)                                    │P(π/16)                                       
q_4 -> 4 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────X──■────────────X───────────────────────────────■─────────────X─────────X───────────────

In [ ]:
# grade_lab1_ex3(pm_staged)

In [ ]:

pm_staged.translation = generate_preset_pass_manager(2, backend=backend).translation
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)


Quantum circuit characteristics
  Depth: 210
  Number of qubits: 10
  Operations: {'rz': 155, 'sx': 10, 'cx': 252}
  Number of multi-qubit Operations: 252


global phase: 1.5723
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───┐     ┌─────────────┐    ┌───┐    ┌────────────┐                       ┌───┐┌────────────┐   ┌───┐    ┌───────────┐                                                   ┌───┐    ┌────────────┐    ┌───┐    ┌───────────┐            ┌───┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
q_0 -> 0 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├─────┤ Rz(-π/1024) ├────┤ X ├────┤ Rz(π/1024) ├───────────────────────┤ X ├┤ Rz(-π/512) ├───┤ X ├────┤ Rz(π/512) ├───────────────────────────────────────────────────┤ X ├────┤ Rz(-π/256) ├────┤ X ├────┤ Rz(π/256) ├─────■──────┤ X ├──────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     

In [ ]:
# grade_lab1_ex4(pm_staged)

In [ ]:
qk_qc = generate_preset_pass_manager(2, backend=backend).run(qc)
print_qc_characteristics(qk_qc)


Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'sx': 10, 'cx': 90}
  Number of multi-qubit Operations: 90


In [ ]:
def noisy_sim(qc, backend):
    # We add measurement operations to the input quantum circuit and then run it on the specified backend
    # A GenericBackendV2 automatically constructs a default model of the expected noise processes,
    # so backend.run would return noisy simulation results
    return backend.run(qc.measure_all(inplace=False), shots=7 * 1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim = noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.2363
Qiskit transpiler fidelity 0.6524


In [ ]:
from qiskit.transpiler.passes import Unroll3qOrMore
from qiskit.transpiler.passes import UnrollCustomDefinitions, BasisTranslator

In [ ]:
# Your work goes here!

pm_opt = StagedPassManager()


# Select pass managers and passes for these five stages as you see suitable, note: not every stage must contain a pass.
# Passes are documented at https://docs.quantum.ibm.com/api/qiskit/transpiler_passes

# Replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_opt.init = generate_unroll_3q(None)
# Initialize the layout stage with an empty pass manager
pm_opt.layout = PassManager()
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout for potential layouting passes
pm_opt.layout += TrivialLayout(coupling_map=cm)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_opt.routing =PassManager(StarPreRouting())
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes


# pm_opt.optimization += ConsolidateBlocks()
# pm_opt.optimization += UnitarySynthesis()
# pm_opt.optimization += CommutativeCancellation()
# pm_opt.optimization += Collect1qRuns()

pm_opt.translation = PassManager([
                                    Collect2qBlocks(),
                                    ConsolidateBlocks(basis_gates=backend.operation_names),
                                    UnitarySynthesis(basis_gates=backend.operation_names)

])
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization for potential optimization passes
pm_opt.optimization = PassManager(Collect1qRuns())


# scheduling is not needed in this lab - we will not run the quantum circuit on a real device
pm_opt.scheduling = None

# do not modify the next line
pm_opt.layout += generate_embed_passmanager(cm)

qc_opt = pm_opt.run(qc)

In [ ]:
# grade your work!
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [ ]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 183
  Number of qubits: 10
  Operations: {'rz': 397, 'sx': 186, 'cx': 133, 'x': 70}
  Number of multi-qubit Operations: 133
Reduction in two-qubit gates compared to qiskit -47.778%!


global phase: 1.5723
           ┌────────┐  ┌────┐  ┌────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           ┌───┐    ┌──────────┐              ┌───┐   ┌────────────┐    ┌───┐     ┌───┐┌─────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
q_0 -> 0 ──┤ Rz(-π) ├──┤ √X ├──┤ Rz(-π) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/2) ├──────────────┤ X ├───┤ Rz(1.5677) ├────┤ X ├─────┤ X ├┤ Rz(-3.1385) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
opt_transpiler_sim = noisy_sim(qc_opt, backend)

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))
print("Own optimized transpiler fidelity", round(hellinger_fidelity(opt_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.2363
Qiskit transpiler fidelity 0.6524
Own optimized transpiler fidelity 0.9157


In [ ]:
class GatesPerQubit(AnalysisPass):
    # Your work here - make sure you implement every abstract method defined in `AnalysisPass` (see https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.AnalysisPass)

   def run(self, dag: DAGCircuit):
        self.property_set["one_q_op"] = {}
        self.property_set["two_q_op"] = {}
        for node in dag.op_nodes():
            if node.op.num_qubits == 1:
                if node.qargs[0] in self.property_set["one_q_op"]:
                    self.property_set["one_q_op"][node.qargs[0]] += 1
                else:
                    self.property_set["one_q_op"][node.qargs[0]] = 1
            elif node.op.num_qubits == 2:
                if node.qargs[0] in self.property_set["two_q_op"]:
                    self.property_set["two_q_op"][node.qargs[0]] += 1
                if node.qargs[0] not in self.property_set["two_q_op"]:
                    self.property_set["two_q_op"][node.qargs[0]] = 1
                if node.qargs[1] in self.property_set["two_q_op"]:
                    self.property_set["two_q_op"][node.qargs[1]] += 1
                if node.qargs[1] not in self.property_set["two_q_op"]:
                    self.property_set["two_q_op"][node.qargs[1]] = 1

In [ ]:
grade_lab1_ex6(GatesPerQubit)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [ ]:
qc = QFT(4, do_swaps=False)
qc = generate_preset_pass_manager(2, backend=backend).run(qc)
gpq = GatesPerQubit()
gpq(qc)
print("single-qubit gates on qubit", gpq.property_set["one_q_op"])
print("two-qubit gates on qubit", gpq.property_set["two_q_op"])
qc.draw(fold=-1, idle_wires=False)

single-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 2): 3, Qubit(QuantumRegister(10, 'q'), 8): 4, Qubit(QuantumRegister(10, 'q'), 0): 6, Qubit(QuantumRegister(10, 'q'), 1): 8}
two-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 2): 6, Qubit(QuantumRegister(10, 'q'), 8): 6, Qubit(QuantumRegister(10, 'q'), 0): 6, Qubit(QuantumRegister(10, 'q'), 1): 6}


global phase: 0
                                                                 ┌───┐    ┌──────────┐   ┌───┐    ┌─────────┐             ┌───┐┌──────────┐            ┌───┐┌──────────┐   ┌────┐   ┌──────────┐                                                              
q_1 -> 0 ────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├─────────────┤ X ├┤ Rz(-π/4) ├────────────┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3π/4) ├─────────────■────────────────■───────────────────────────────
                                                                 └─┬─┘    └──────────┘   └─┬─┘    └──┬───┬──┘┌───────────┐└─┬─┘└──┬───┬───┘┌──────────┐└─┬─┘└──┬───┬───┘┌──┴────┴──┐└──┬───┬───┘┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐┌────┐┌─────────┐
q_0 -> 1 ──────────────────────────────────────────────────────────┼───────────────────────┼─────────┤ X ├───┤ Rz(-π/16) ├──┼─────┤ X ├────┤ Rz(π/16) ├──┼─────┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
         ┌─────────┐┌────┐┌────────────┐                           │                       │         └─┬─┘   └───────────┘  │     └─┬─┘    └──────────┘  │     └─┬─┘    └──────────┘   └─┬─┘    └─────────┘└───┘└──────────┘└───┘└──────────┘└────┘└─────────┘
q_3 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(15π/16) ├──■────────────────■───────■───────────────────────■───────────■────────────────────┼───────■────────────────────┼───────┼───────────────────────┼────────────────────────────────────────────────────────────────────
         └─────────┘└────┘└────────────┘┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐   ┌────┐   ┌──────────┐                          │                            │       │                       │                                                                    
q_2 -> 8 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(7π/8) ├──────────────────────────■────────────────────────────■───────■───────────────────────■────────────────────────────────────────────────────────────────────
                                        └───┘└──────────┘└───┘└──────────┘   └────┘   └──────────┘

In [ ]:
pg = Gate('Peres', 3, params=[], label='PG')

In [ ]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

┌─────┐
q_0: ┤0    ├
     │     │
q_1: ┤1 PG ├
     │     │
q_2: ┤2    ├
     └─────┘

In [ ]:
def get_qc_in(nq):
    # QFT circuit, feel free to use a previously defined pass manager for the QFT circuit
    qc_qft = QFT(nq, do_swaps=False)
    # part of the circuit including the Peres gate
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    # add QFT circuit to qc_in
    qc_in.compose(qc_qft, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq - 1)
    # add circuit with peres gates
    qc_in.compose(qc_inner, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    # add inverse QFT circuit
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)

┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [ ]:
class PeresGateTranslation(TransformationPass):
    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec

    # Your work here - make sure you implement every abstract method defined by https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.TransformationPass

    def call(self, qc):
        self.run(circuit_to_dag(qc))

    def run(self, dag):
        for node in dag.op_nodes():
            if node.name=='Peres':
                dag.substitute_node_with_dag(node, circuit_to_dag(self.get_peres_decomposition()))

        return dag

In [ ]:
grade_lab1_ex7(PeresGateTranslation, pg)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.
